# Convert OpenSfM reconstruction to format used by our tool

In [37]:
import sys
import json
import pickle
import numpy as np
import cv2
import networkx as nx
import matplotlib.pyplot as plt

sys.path.append("..")
from mapper.modules import triangulate_modules
from mapper.map_points import MapPoints

In [38]:
reconstructions = json.load(open("reconstruction.json", "r"))

print("Number of reconstructions: {}".format(len(reconstructions)))

Number of reconstructions: 1


In [39]:
reconstruction = reconstructions[0]

### Convert map points

In [40]:
map_points = MapPoints()
num_points = len(reconstruction['points'])
map_points.pts_3d = np.zeros((num_points, 3), dtype=np.float64)
for i, point in enumerate(reconstruction['points'].values()):
    map_points.pts_3d[i, :] = point['coordinates']

### Convert camera poses

In [41]:
pose_graph = nx.Graph()
for i, frame_name in enumerate(sorted(reconstruction['shots'])):
    shot = reconstruction['shots'][frame_name]
    R, _ = cv2.Rodrigues(np.array(shot["rotation"]))
    t = np.array(shot["translation"])
    t = -R.T.dot(t)
    R, _ = cv2.Rodrigues(R.T)
    pose = np.hstack((R.reshape(3,), t))
    frame_name = str.split(frame_name, ".")[0]
    pose_graph.add_node(i, frame_name=frame_name, pose=pose)

In [42]:
pickle.dump(map_points, open("map_points.pkl", "wb"))
pickle.dump(pose_graph, open("pose_graph.pkl", "wb"))